# Assemble Data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.ndimage import uniform_filter1d  # simple smoothing
import re

In [2]:
pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_rows', None)  # show all columns
import os
os.chdir('..')

### Start with plot features

In [3]:
# get elevation features
df = pd.read_pickle('data/plot_elev_features.pkl')

### Compute vectors for aspect and slope, and some interaction terms

In [4]:
# Aspect: convert to radians and compute sin/cos
df['aspect_min_cos'] = np.cos(np.radians(df['aspect_min']))
df['aspect_min_sin'] = np.sin(np.radians(df['aspect_min']))

df['aspect_max_cos'] = np.cos(np.radians(df['aspect_max']))
df['aspect_max_sin'] = np.sin(np.radians(df['aspect_max']))

df['aspect_mean_cos'] = np.cos(np.radians(df['aspect_mean']))
df['aspect_mean_sin'] = np.sin(np.radians(df['aspect_mean']))

# Drop raw aspect values
df = df.drop(columns=['aspect_min', 'aspect_max', 'aspect_mean'])

df['slope_x'] = df['slope_mean'] * df['aspect_mean_cos']
df['slope_y'] = df['slope_mean'] * df['aspect_mean_sin']

## Now add NDVI for each plot to features

### open up the filtered and smoothed ndvi df

In [5]:
veg_agg = pd.read_pickle('data/ndvi/plots/final_df.pkl')
keep_cols = ['plot_id', 'year']

mean_cols = [col for col in veg_agg.columns 
             if '_mean_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
keep_cols.extend(mean_cols)

slope_cols = [col for col in veg_agg.columns 
              if '_slope_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
keep_cols.extend(slope_cols)

std_cols = [col for col in veg_agg.columns 
            if '_std_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
keep_cols.extend(std_cols)

sos_cols = [col for col in veg_agg.columns if '_sos_month' in col]
keep_cols.extend(sos_cols)

season_length_cols = [col for col in veg_agg.columns if '_season_length' in col]
keep_cols.extend(season_length_cols)

senescence_cols = [col for col in veg_agg.columns if '_senescence_rate' in col]
keep_cols.extend(senescence_cols)

veg_agg = veg_agg[keep_cols].copy()

In [6]:
veg_agg = veg_agg.dropna()
veg_agg

,plot_id,year,evi_smooth_mean_4,evi_smooth_mean_5,evi_smooth_mean_6,evi_smooth_mean_7,evi_smooth_mean_8,evi_smooth_mean_9,evi_smooth_mean_10,mcari2_smooth_mean_4,mcari2_smooth_mean_5,mcari2_smooth_mean_6,mcari2_smooth_mean_7,mcari2_smooth_mean_8,mcari2_smooth_mean_9,mcari2_smooth_mean_10,ndvi_smooth_mean_4,ndvi_smooth_mean_5,ndvi_smooth_mean_6,ndvi_smooth_mean_7,ndvi_smooth_mean_8,ndvi_smooth_mean_9,ndvi_smooth_mean_10,ndwi_smooth_mean_4,ndwi_smooth_mean_5,ndwi_smooth_mean_6,ndwi_smooth_mean_7,ndwi_smooth_mean_8,ndwi_smooth_mean_9,ndwi_smooth_mean_10,rendvi_smooth_mean_4,rendvi_smooth_mean_5,rendvi_smooth_mean_6,rendvi_smooth_mean_7,rendvi_smooth_mean_8,rendvi_smooth_mean_9,rendvi_smooth_mean_10,savi_smooth_mean_4,savi_smooth_mean_5,savi_smooth_mean_6,savi_smooth_mean_7,savi_smooth_mean_8,savi_smooth_mean_9,savi_smooth_mean_10,evi_smooth_slope_4,evi_smooth_slope_5,evi_smooth_slope_6,evi_smooth_slope_7,evi_smooth_slope_8,evi_smooth_slope_9,evi_smooth_slope_10,mcari2_smooth_slope_4,mcari2_smooth_slope_5,mcari2_smooth_slope_6,mcari2_smooth_slope_7,mcari2_smooth_slope_8,mcari2_smooth_slope_9,mcari2_smooth_slope_10,ndvi_smooth_slope_4,ndvi_smooth_slope_5,ndvi_smooth_slope_6,ndvi_smooth_slope_7,ndvi_smooth_slope_8,ndvi_smooth_slope_9,ndvi_smooth_slope_10,ndwi_smooth_slope_4,ndwi_smooth_slope_5,ndwi_smooth_slope_6,ndwi_smooth_slope_7,ndwi_smooth_slope_8,ndwi_smooth_slope_9,ndwi_smooth_slope_10,rendvi_smooth_slope_4,rendvi_smooth_slope_5,rendvi_smooth_slope_6,rendvi_smooth_slope_7,rendvi_smooth_slope_8,rendvi_smooth_slope_9,rendvi_smooth_slope_10,savi_smooth_slope_4,savi_smooth_slope_5,savi_smooth_slope_6,savi_smooth_slope_7,savi_smooth_slope_8,savi_smooth_slope_9,savi_smooth_slope_10,evi_smooth_std_4,evi_smooth_std_5,evi_smooth_std_6,evi_smooth_std_7,evi_smooth_std_8,evi_smooth_std_9,evi_smooth_std_10,mcari2_smooth_std_4,mcari2_smooth_std_5,mcari2_smooth_std_6,mcari2_smooth_std_7,mcari2_smooth_std_8,mcari2_smooth_std_9,mcari2_smooth_std_10,ndvi_smooth_std_4,ndvi_smooth_std_5,ndvi_smooth_std_6,ndvi_smooth_std_7,ndvi_smooth_std_8,ndvi_smooth_std_9,ndvi_smooth_std_10,ndwi_smooth_std_4,ndwi_smooth_std_5,ndwi_smooth_std_6,ndwi_smooth_std_7,ndwi_smooth_std_8,ndwi_smooth_std_9,ndwi_smooth_std_10,rendvi_smooth_std_4,rendvi_smooth_std_5,rendvi_smooth_std_6,rendvi_smooth_std_7,rendvi_smooth_std_8,rendvi_smooth_std_9,rendvi_smooth_std_10,savi_smooth_std_4,savi_smooth_std_5,savi_smooth_std_6,savi_smooth_std_7,savi_smooth_std_8,savi_smooth_std_9,savi_smooth_std_10,ndvi_smooth_sos_month,evi_smooth_sos_month,ndwi_smooth_sos_month,savi_smooth_sos_month,rendvi_smooth_sos_month,mcari2_smooth_sos_month,ndvi_smooth_season_length,evi_smooth_season_length,ndwi_smooth_season_length,savi_smooth_season_length,rendvi_smooth_season_length,mcari2_smooth_season_length,ndvi_smooth_senescence_rate,evi_smooth_senescence_rate,ndwi_smooth_senescence_rate,savi_smooth_senescence_rate,rendvi_smooth_senescence_rate,mcari2_smooth_senescence_rate
2,0,2018,0.671277,0.817120,1.065970,1.022595,1.066351,0.821679,0.797427,941.515499,1468.287845,2448.610024,2391.271378,2106.199943,1496.124878,1123.917446,0.306933,0.382007,0.490689,0.469544,0.436076,0.414156,0.400434,-0.007614,0.022456,0.090046,0.083468,0.060841,0.024533,0.020689,0.193792,0.244679,0.325610,0.331031,0.304348,0.260505,0.227853,0.460341,0.572945,0.735949,0.704234,0.654038,0.621156,0.600573,0.016984,0.006030,0.008733,-0.006656,0.000496,-0.005541,0.006710,44.535986,29.396829,31.357586,-19.610372,-2.045001,-17.392529,-0.452437,0.007096,0.003574,0.003159,-0.002362,0.000593,-0.001004,0.001247,0.000941,0.002322,0.001876,-0.001337,-0.000301,-0.000881,0.001153,0.004689,0.002377,0.002691,-0.001681,0.000121,-0.001545,0.000518,0.010643,0.005360,0.004737,-0.003543,0.000889,-0.001506,0.001871,0.036030,0.049537,0.074757,0.058198,0.056271,0.075031,0.044868,94.475093,234.720820,264.468617,173.030001,75.486726,220.929803,55.157906,0.015053,0.028276,0.026889,0.020678,0.007869,0.012650,0.009231,0.001996,0.018297,0.015756,0.011729,0.00

In [7]:
df = df.merge(veg_agg, how = 'inner', on = 'plot_id')

### Now let's add some weather information

First load the wather that has been unzipped and clipped to the vineyard

In [8]:
weather = pd.read_pickle('data/PRISM/df.pkl')



weather = (
    weather
    .groupby("date", as_index=False)
    .first()
)


weather['date'] = pd.to_datetime(weather['date'])
weather['doy'] = weather['date'].dt.dayofyear
weather['year'] = weather['date'].dt.year

weather

,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year
0,2016-01-01,1,0.0000,-0.415000,-3.153000,-5.8920,2.192000,0.4860,1,2016
1,2016-01-02,1,0.0000,-4.138000,-5.380000,-6.6220,1.168000,0.5970,2,2016
2,2016-01-03,1,0.0000,-4.943000,-6.424000,-7.9050,1.166000,0.5900,3,2016
3,2016-01-04,1,1.0160,-4.861000,-6.448000,-8.0370,0.841000,0.2900,4,2016
4,2016-01-05,1,0.0000,-2.711000,-4.315000,-5.9210,1.130000,0.2980,5,2016
...,...,...,...,...,...,...,...,...,...,...
3550,2025-09-20,1,0.0000,30.182299,21.725849,13.2694,26.377600,0.8048,263,2025
3551,2025-09-21,1,0.0000,32.516800,23.064949,13.6131,41.111298,2.2150,264,2025
3552,2025-09-22,1,1.5994,22.928999,15.978449,9.0279,14.408899,0.3890,265,2025
3553,2025-09-23,1,0.0000,22.489500,15.037100,7.5847,15.716000,0.2265,266,2025


### Crack at some frost and growing degree days cumulative per month

In [9]:
# Compute frost days and GDD
weather['frost'] = (weather['tmin'] < 0)
weather['gdd'] = (weather['tmean'] - 10).clip(lower=0) 

# disregard 2025 for now
weather = weather[weather['year'] != 2025].copy()

In [10]:
# Compute cumulative GDD for each month
weather['month'] = weather['date'].dt.month
cumulative_gdd = weather.groupby([ 'year', 'month'])['gdd'].sum().groupby(level=[0,1]).cumsum()
# Reset index to turn MultiIndex into columns
cumulative_gdd = cumulative_gdd.rename('cumulative_gdd').reset_index()

# Merge back to weather
weather = weather.merge(
    cumulative_gdd,
    on=['year', 'month'],
    how='left'
)


weather

,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year,frost,gdd,month,cumulative_gdd
0,2016-01-01,1,0.0000,-0.415000,-3.15300,-5.8920,2.1920,0.4860,1,2016,True,0.0,1,0.0
1,2016-01-02,1,0.0000,-4.138000,-5.38000,-6.6220,1.1680,0.5970,2,2016,True,0.0,1,0.0
2,2016-01-03,1,0.0000,-4.943000,-6.42400,-7.9050,1.1660,0.5900,3,2016,True,0.0,1,0.0
3,2016-01-04,1,1.0160,-4.861000,-6.44800,-8.0370,0.8410,0.2900,4,2016,True,0.0,1,0.0
4,2016-01-05,1,0.0000,-2.711000,-4.31500,-5.9210,1.1300,0.2980,5,2016,True,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2024-12-27,1,5.5226,12.897300,8.57640,4.2555,5.3272,0.0464,362,2024,False,0.0,12,0.0
3284,2024-12-28,1,2.7350,9.500899,6.92395,4.3470,1.8989,0.0603,363,2024,False,0.0,12,0.0
3285,2024-12-29,1,6.4769,13.031700,9.00255,4.9734,4.6196,0.0594,364,2024,False,0.0,12,0.0
3286,2024-12-30,1,0.9107,9.572500,6.35390,3.1353,2.6949,0.1705,365,2024,False,0.0,12,0.0


In [11]:
# # compute seasonal weather


# early = (weather['doy'] <= 100)        # roughly day 1-100
# mid = (weather['doy'] > 100) & (weather['doy'] <= 200)
# late = (weather['doy'] > 200)

# # Compute total precipitation per season for each plot/year
# weather_seasonal = weather.groupby(['year']).apply(
#     lambda x: pd.Series({
#         'ppt_early': x.loc[early, 'ppt'].sum(),
#         'ppt_mid': x.loc[mid, 'ppt'].sum(),
#         'ppt_late': x.loc[late, 'ppt'].sum(),
        
#         'tmax_early_mean': x.loc[early, 'tmax'].mean(),
#         'tmax_mid_mean': x.loc[mid, 'tmax'].mean(),
#         'tmax_late_mean': x.loc[late, 'tmax'].mean(),
#         'tmax_early_min': x.loc[early, 'tmax'].min(),
#         'tmax_mid_min': x.loc[mid, 'tmax'].min(),
#         'tmax_late_min': x.loc[late, 'tmax'].min(),
#         'tmax_early_max': x.loc[early, 'tmax'].max(),
#         'tmax_mid_max': x.loc[mid, 'tmax'].max(),
#         'tmax_late_max': x.loc[late, 'tmax'].max(),
        
#         'tmin_early_mean': x.loc[early, 'tmin'].mean(),
#         'tmin_mid_mean': x.loc[mid, 'tmin'].mean(),
#         'tmin_late_mean': x.loc[late, 'tmin'].mean(),
#         'tmin_early_min': x.loc[early, 'tmin'].min(),
#         'tmin_mid_min': x.loc[mid, 'tmin'].min(),
#         'tmin_late_min': x.loc[late, 'tmin'].min(),
#         'tmin_early_max': x.loc[early, 'tmin'].max(),
#         'tmin_mid_max': x.loc[mid, 'tmin'].max(),
#         'tmin_late_max': x.loc[late, 'tmin'].max(),
        
#         'tmean_early_mean': x.loc[early, 'tmean'].mean(),
#         'tmean_mid_mean': x.loc[mid, 'tmean'].mean(),
#         'tmean_late_mean': x.loc[late, 'tmean'].mean(),
#         'tmean_early_min': x.loc[early, 'tmean'].min(),
#         'tmean_mid_min': x.loc[mid, 'tmean'].min(),
#         'tmean_late_min': x.loc[late, 'tmean'].min(),
#         'tmean_early_max': x.loc[early, 'tmean'].max(),
#         'tmean_mid_max': x.loc[mid, 'tmean'].max(),
#         'tmean_late_max': x.loc[late, 'tmean'].max(),
        
#         'vpdmax_early_mean': x.loc[early, 'vpdmax'].mean(),
#         'vpdmax_mid_mean': x.loc[mid, 'vpdmax'].mean(),
#         'vpdmax_late_mean': x.loc[late, 'vpdmax'].mean(),
#         'vpdmax_early_min': x.loc[early, 'vpdmax'].min(),
#         'vpdmax_mid_min': x.loc[mid, 'vpdmax'].min(),
#         'vpdmax_late_min': x.loc[late, 'vpdmax'].min(),
#         'vpdmax_early_max': x.loc[early, 'vpdmax'].max(),
#         'vpdmax_mid_max': x.loc[mid, 'vpdmax'].max(),
#         'vpdmax_late_max': x.loc[late, 'vpdmax'].max(),
        
#         'vpdmin_early_mean': x.loc[early, 'vpdmin'].mean(),
#         'vpdmin_mid_mean': x.loc[mid, 'vpdmin'].mean(),
#         'vpdmin_late_mean': x.loc[late, 'vpdmin'].mean(),
#         'vpdmin_early_min': x.loc[early, 'vpdmin'].min(),
#         'vpdmin_mid_min': x.loc[mid, 'vpdmin'].min(),
#         'vpdmin_late_min': x.loc[late, 'vpdmin'].min(),
#         'vpdmin_early_max': x.loc[early, 'vpdmin'].max(),
#         'vpdmin_mid_max': x.loc[mid, 'vpdmin'].max(),
#         'vpdmin_late_max': x.loc[late, 'vpdmin'].max(),
        
#     })
# ).reset_index()
# weather_seasonal

In [12]:
# Compute mean/std per feature over historical years
climate_features = ['ppt', 'tmean', 'tmax','tmin','vpdmax','vpdmin', 'gdd']

for feat in climate_features:
    mean_hist = weather.groupby('year')[feat].transform('mean')
    std_hist = weather.groupby('year')[feat].transform('std')
    
    # z-score relative to historical mean/std
    weather[f'{feat}_anom'] = (weather[feat] - mean_hist) / std_hist

In [13]:
# Aggregate by year and month
weather_agg = weather.groupby(['year','month']).agg(
    ppt_sum=('ppt', 'sum'),
    ppt_max=('ppt','max'),
    ppt_avg=('ppt','mean'),
    ppt_var=('ppt','var'),
    tmean_max=('tmean','max'),
    tmean_min=('tmean', 'min'),
    tmean_avg=('tmean', 'mean'),
    tmean_var=('tmean','var'),
    tmax_max=('tmax','max'),
    tmax_min=('tmax', 'min'),
    tmax_mean=('tmax', 'mean'),
    tmax_var=('tmax','var'),
    tmin_max=('tmin','max'),
    tmin_min=('tmin', 'min'),
    tmin_avg=('tmin', 'mean'),
    tmin_var=('tmin','var'),

    vpdmax_mean=('vpdmax', 'mean'),
    vpdmax_max=('vpdmax','max'),
    vpdmax_min=('vpdmax','min'),
    vpdmax_var=('vpdmax','var'),

    vpdmin_mean=('vpdmin', 'mean'),
    vpdmin_max=('vpdmin','max'),
    vpdmin_min=('vpdmin','min'),
    vpdmin_var=('vpdmin','var'),    
    frost_days=('frost', 'sum'),
    gdd_sum=('gdd', 'sum')
).reset_index()

weather_agg

,year,month,ppt_sum,ppt_max,ppt_avg,ppt_var,tmean_max,tmean_min,tmean_avg,tmean_var,tmax_max,tmax_min,tmax_mean,tmax_var,tmin_max,tmin_min,tmin_avg,tmin_var,vpdmax_mean,vpdmax_max,vpdmax_min,vpdmax_var,vpdmin_mean,vpdmin_max,vpdmin_min,vpdmin_var,frost_days,gdd_sum
0,2016,1,39.108000,11.5400,1.261548,6.837618,9.109000,-6.448000,2.363968,17.294123,14.153000,-4.943000,5.021710,26.001322,4.605000,-8.0370,-0.293129,12.186422,2.393387,6.897000,0.458000,3.780401,0.189742,0.5970,0.0160,0.039542,13.0,0.000000
1,2016,2,11.718000,2.7090,0.404069,0.593875,12.482000,1.593000,6.453897,7.680867,17.299999,6.118000,11.750965,7.397104,7.666000,-2.9350,1.157621,10.340643,6.104000,11.297000,2.866000,4.768977,0.277276,1.4740,0.0400,0.096766,14.0,5.089003
2,2016,3,20.291000,4.9090,0.654548,1.547398,12.769001,5.576000,8.575258,2.796051,19.795000,11.013000,14.400323,5.355179,8.587000,-1.2010,2.751161,4.416925,9.227968,17.035000,4.144000,9.286262,0.588968,1.9800,0.0520,0.208050,2.0,5.973003
3,2016,4,10.538000,5.4370,0.351267,1.388352,18.854000,10.159000,14.469167,5.952204,29.745001,15.809000,22.134167,15.206971,11.151000,3.0690,6.805067,4.698607,18.558433,30.785000,10.264000,30.101468,1.313200,3.7120,0.3560,0.507230,0.0,134.075020
4,2016,5,21.534000,13.5400,0.694645,6.692758,22.754002,12.478001,17.206517,6.196761,31.068001,16.572001,24.594226,14.913069,14.742000,5.5850,9.819871,5.052538,22.228387,37.235001,5.905000,60.213045,2.090129,5.7750,0.3800,1.890084,0.0,223.402028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2024,8,0.000000,0.0000,0.000000,0.000000,30.093048,15.248400,22.754270,15.593661,40.419498,20.771200,31.101335,20.853033,22.065800,7.0781,14.407206,14.503936,33.703335,59.763298,12.236600,118.870091,3.640297,8.1939,0.8798,4.481632,0.0,395.382382
104,2024,9,1.944400,1.1703,0.064813,0.054936,25.681200,13.459349,20.427579,10.544275,34.972099,22.057400,28.527736,18.324130,17.259699,4.7870,12.327423,8.797355,28.967986,44.213699,14.604199,78.403110,2.876540,6.7950,1.1210,1.933615,0.0,312.827381
105,2024,10,8.102000,4.9673,0.261355,0.829859,17.374100,5.547200,12.222459,10.453544,27.873999,11.774500,19.772248,17.658261,9.920400,-0.6801,4.672671,7.522042,14.089916,27.932299,6.162100,25.024306,0.579826,2.0530,0.1139,0.184995,2.0,85.815940
106,2024,11,49.009998,9.9625,1.633667,6.608761,11.301700,1.106600,6.645866,7.511766,16.026400,2.173600,10.713420,10.542870,8.142900,-1.3513,2.578313,8.497044,4.277200,10.192400,0.387600,5.935451,0.232310,1.2538,0.0197,0.075708,7.0,3.364398


In [14]:
weather_agg = weather_agg.drop(weather_agg[weather_agg['month'].isin([1,2,3,11,12])].index)

### Widen the data to remove months dependency

In [15]:
weather_wide = weather_agg.pivot_table(
    index=['year'],
    columns='month',
    values=weather_agg.columns[2:],  # list all weather vars
).reset_index()
# Flatten multi-index columns
weather_wide.columns = ['_'.join([str(c) for c in col if c != '']) for col in weather_wide.columns]

In [16]:
weather_wide

,year,frost_days_4,frost_days_5,frost_days_6,frost_days_7,frost_days_8,frost_days_9,frost_days_10,gdd_sum_4,gdd_sum_5,gdd_sum_6,gdd_sum_7,gdd_sum_8,gdd_sum_9,gdd_sum_10,ppt_avg_4,ppt_avg_5,ppt_avg_6,ppt_avg_7,ppt_avg_8,ppt_avg_9,ppt_avg_10,ppt_max_4,ppt_max_5,ppt_max_6,ppt_max_7,ppt_max_8,ppt_max_9,ppt_max_10,ppt_sum_4,ppt_sum_5,ppt_sum_6,ppt_sum_7,ppt_sum_8,ppt_sum_9,ppt_sum_10,ppt_var_4,ppt_var_5,ppt_var_6,ppt_var_7,ppt_var_8,ppt_var_9,ppt_var_10,tmax_max_4,tmax_max_5,tmax_max_6,tmax_max_7,tmax_max_8,tmax_max_9,tmax_max_10,tmax_mean_4,tmax_mean_5,tmax_mean_6,tmax_mean_7,tmax_mean_8,tmax_mean_9,tmax_mean_10,tmax_min_4,tmax_min_5,tmax_min_6,tmax_min_7,tmax_min_8,tmax_min_9,tmax_min_10,tmax_var_4,tmax_var_5,tmax_var_6,tmax_var_7,tmax_var_8,tmax_var_9,tmax_var_10,tmean_avg_4,tmean_avg_5,tmean_avg_6,tmean_avg_7,tmean_avg_8,tmean_avg_9,tmean_avg_10,tmean_max_4,tmean_max_5,tmean_max_6,tmean_max_7,tmean_max_8,tmean_max_9,tmean_max_10,tmean_min_4,tmean_min_5,tmean_min_6,tmean_min_7,tmean_min_8,tmean_min_9,tmean_min_10,tmean_var_4,tmean_var_5,tmean_var_6,tmean_var_7,tmean_var_8,tmean_var_9,tmean_var_10,tmin_avg_4,tmin_avg_5,tmin_avg_6,tmin_avg_7,tmin_avg_8,tmin_avg_9,tmin_avg_10,tmin_max_4,tmin_max_5,tmin_max_6,tmin_max_7,tmin_max_8,tmin_max_9,tmin_max_10,tmin_min_4,tmin_min_5,tmin_min_6,tmin_min_7,tmin_min_8,tmin_min_9,tmin_min_10,tmin_var_4,tmin_var_5,tmin_var_6,tmin_var_7,tmin_var_8,tmin_var_9,tmin_var_10,vpdmax_max_4,vpdmax_max_5,vpdmax_max_6,vpdmax_max_7,vpdmax_max_8,vpdmax_max_9,vpdmax_max_10,vpdmax_mean_4,vpdmax_mean_5,vpdmax_mean_6,vpdmax_mean_7,vpdmax_mean_8,vpdmax_mean_9,vpdmax_mean_10,vpdmax_min_4,vpdmax_min_5,vpdmax_min_6,vpdmax_min_7,vpdmax_min_8,vpdmax_min_9,vpdmax_min_10,vpdmax_var_4,vpdmax_var_5,vpdmax_var_6,vpdmax_var_7,vpdmax_var_8,vpdmax_var_9,vpdmax_var_10,vpdmin_max_4,vpdmin_max_5,vpdmin_max_6,vpdmin_max_7,vpdmin_max_8,vpdmin_max_9,vpdmin_max_10,vpdmin_mean_4,vpdmin_mean_5,vpdmin_mean_6,vpdmin_mean_7,vpdmin_mean_8,vpdmin_mean_9,vpdmin_mean_10,vpdmin_min_4,vpdmin_min_5,vpdmin_min_6,vpdmin_min_7,vpdmin_min_8,vpdmin_min_9,vpdmin_min_10,vpdmin_var_4,vpdmin_var_5,vpdmin_var_6,vpdmin_var_7,vpdmin_var_8,vpdmin_var_9,vpdmin_var_10
0,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134.075020,223.402028,307.660027,405.618038,407.439035,235.476027,76.512014,0.351267,0.694645,0.116267,0.336129,0.052774,0.188567,1.522871,5.4370,13.5400,1.322000,7.5470,1.6360,4.3250,7.2630,10.538000,21.534000,3.488000,10.4200,1.6360,5.6570,47.209000,1.388352,6.692758,0.129404,1.869916,0.086339,0.669418,4.140515,29.745001,31.068001,38.674000,38.367001,36.757000,32.688999,24.534000,22.134167,24.594226,28.647133,30.991258,31.818709,25.375433,17.185742,15.8090,16.572001,20.188000,23.725000,25.487000,19.135000,10.654000,15.206971,14.913069,30.450390,14.327391,11.325470,10.323048,11.537235,14.469167,17.206517,20.255334,23.084453,23.143195,17.849201,12.265710,18.854000,22.754002,28.107002,28.642002,27.079000,21.796001,17.392000,10.159000,12.478001,13.744000,18.933001,19.149000,14.357000,8.353001,5.952204,6.196761,19.160403,6.567869,5.256573,4.180680,5.974943,6.805067,9.819871,11.864600,15.178710,14.468774,10.323867,7.346742,11.151000,14.742000,18.356001,18.965000,18.174000,16.412001,10.8610,3.0690,5.5850,6.5000,12.668000,11.484000,5.9150,1.4650,4.698607,5.052538,12.786345,3.237144,2.809082,5.767369,5.246719,30.785000,37.235001,57.108002,54.015999,51.394001,39.612999,21.930000,18.558433,22.228387,31.716133,34.502387,37.469806,23.103567,9.629839,10.2640,5.9050,16.652000,18.617001,21.361000,11.226000,1.3780,30.101468,60.213045,154.416290,103.459511,79.107913,45.414433,29.330386,3.7120,5.775000,6.116000,7.134000,6.9110,6.4880,3.4350,1.313200,2.090129,2.933700,3.553032,3.681161,2.030733,0.668226,0.3560,0.3800,0.683000,0.9460,1.523000,0.6290,0.0680,0.507230,1.890084,2.225213,1.893454,1.513597,1.311729,0.531030
1,2017,0.0,0.0,0.0,0.0,0.0,0.0,2.0,30.905011,182.014022,291.664028,456.326035,449.380043,266.912029,47.258013,1.348867,0.229677,0.080200,0.000000,0.082065,0.156900,

### Now we can combine data.

In [17]:
df['year'] = df['year'].astype(int)
df = pd.merge(df, weather_wide, how = 'inner', on = 'year')

In [18]:
df

,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_x,slope_y,year,evi_smooth_mean_4,evi_smooth_mean_5,evi_smooth_mean_6,evi_smooth_mean_7,evi_smooth_mean_8,evi_smooth_mean_9,evi_smooth_mean_10,mcari2_smooth_mean_4,mcari2_smooth_mean_5,mcari2_smooth_mean_6,mcari2_smooth_mean_7,mcari2_smooth_mean_8,mcari2_smooth_mean_9,mcari2_smooth_mean_10,ndvi_smooth_mean_4,ndvi_smooth_mean_5,ndvi_smooth_mean_6,ndvi_smooth_mean_7,ndvi_smooth_mean_8,ndvi_smooth_mean_9,ndvi_smooth_mean_10,ndwi_smooth_mean_4,ndwi_smooth_mean_5,ndwi_smooth_mean_6,ndwi_smooth_mean_7,ndwi_smooth_mean_8,ndwi_smooth_mean_9,ndwi_smooth_mean_10,rendvi_smooth_mean_4,rendvi_smooth_mean_5,rendvi_smooth_mean_6,rendvi_smooth_mean_7,rendvi_smooth_mean_8,rendvi_smooth_mean_9,rendvi_smooth_mean_10,savi_smooth_mean_4,savi_smooth_mean_5,savi_smooth_mean_6,savi_smooth_mean_7,savi_smooth_mean_8,savi_smooth_mean_9,savi_smooth_mean_10,evi_smooth_slope_4,evi_smooth_slope_5,evi_smooth_slope_6,evi_smooth_slope_7,evi_smooth_slope_8,evi_smooth_slope_9,evi_smooth_slope_10,mcari2_smooth_slope_4,mcari2_smooth_slope_5,mcari2_smooth_slope_6,mcari2_smooth_slope_7,mcari2_smooth_slope_8,mcari2_smooth_slope_9,mcari2_smooth_slope_10,ndvi_smooth_slope_4,ndvi_smooth_slope_5,ndvi_smooth_slope_6,ndvi_smooth_slope_7,ndvi_smooth_slope_8,ndvi_smooth_slope_9,ndvi_smooth_slope_10,ndwi_smooth_slope_4,ndwi_smooth_slope_5,ndwi_smooth_slope_6,ndwi_smooth_slope_7,ndwi_smooth_slope_8,ndwi_smooth_slope_9,ndwi_smooth_slope_10,rendvi_smooth_slope_4,rendvi_smooth_slope_5,rendvi_smooth_slope_6,rendvi_smooth_slope_7,rendvi_smooth_slope_8,rendvi_smooth_slope_9,rendvi_smooth_slope_10,savi_smooth_slope_4,savi_smooth_slope_5,savi_smooth_slope_6,savi_smooth_slope_7,savi_smooth_slope_8,savi_smooth_slope_9,savi_smooth_slope_10,evi_smooth_std_4,evi_smooth_std_5,evi_smooth_std_6,evi_smooth_std_7,evi_smooth_std_8,evi_smooth_std_9,evi_smooth_std_10,mcari2_smooth_std_4,mcari2_smooth_std_5,mcari2_smooth_std_6,mcari2_smooth_std_7,mcari2_smooth_std_8,mcari2_smooth_std_9,mcari2_smooth_std_10,ndvi_smooth_std_4,ndvi_smooth_std_5,ndvi_smooth_std_6,ndvi_smooth_std_7,ndvi_smooth_std_8,ndvi_smooth_std_9,ndvi_smooth_std_10,ndwi_smooth_std_4,ndwi_smooth_std_5,ndwi_smooth_std_6,ndwi_smooth_std_7,ndwi_smooth_std_8,ndwi_smooth_std_9,ndwi_smooth_std_10,rendvi_smooth_std_4,rendvi_smooth_std_5,rendvi_smooth_std_6,rendvi_smooth_std_7,rendvi_smooth_std_8,rendvi_smooth_std_9,rendvi_smooth_std_10,savi_smooth_std_4,savi_smooth_std_5,savi_smooth_std_6,savi_smooth_std_7,savi_smooth_std_8,savi_smooth_std_9,savi_smooth_std_10,ndvi_smooth_sos_month,evi_smooth_sos_month,ndwi_smooth_sos_month,savi_smooth_sos_month,rendvi_smooth_sos_month,mcari2_smooth_sos_month,ndvi_smooth_season_length,evi_smooth_season_length,ndwi_smooth_season_length,savi_smooth_season_length,rendvi_smooth_season_length,mcari2_smooth_season_length,ndvi_smooth_senescence_rate,evi_smooth_senescence_rate,ndwi_smooth_senescence_rate,savi_smooth_senescence_rate,rendvi_smooth_senescence_rate,mcari2_smooth_senescence_rate,frost_days_4,frost_days_5,frost_days_6,frost_days_7,frost_days_8,frost_days_9,frost_days_10,gdd_sum_4,gdd_sum_5,gdd_sum_6,gdd_sum_7,gdd_sum_8,gdd_sum_9,gdd_sum_10,ppt_avg_4,ppt_avg_5,ppt_avg_6,ppt_avg_7,ppt_avg_8,ppt_avg_9,ppt_avg_10,ppt_max_4,ppt_max_5,ppt_max_6,ppt_max_7,ppt_max_8,ppt_max_9,ppt_max_10,ppt_sum_4,ppt_sum_5,ppt_sum_6,ppt_sum_7,ppt_sum_8,ppt_sum_9,ppt_sum_10,ppt_var_4,ppt_var_5,ppt_var_6,ppt_var_7,ppt_var_8,ppt_var_9,ppt_var_10,tmax_max_4,tmax_max_5,tmax_max_6,tmax_max_7,tmax_max_8,tmax_max_9,tmax_max_10,tmax_mean_4,tmax_mean_5,tmax_mean_6,tmax_mean_7,tmax_mean_8,tmax_mean_9,tmax_mean_10,tmax_min_4,tmax_min_5,tmax_min_6,tmax_min_7,tmax_min_8,tmax_min_9,tmax_

In [19]:
# # Optional: could also add slope magnitude transforms
# df['slope_squared'] = df['slope_mean'] ** 2
# df['slope_log'] = np.log1p(df['slope_mean'])  # log(1 + slope)

for i in range(4, 11, 1):
    
    df[f'water_availability_{i}'] = df[f'ppt_sum_{i}'] / (1 +  df[f'gdd_sum_{i}'])
    df[f'diurnal_temp_range_{i}'] = df[f'tmax_mean_{i}'] / df[f'tmin_avg_{i}']
    df[f'stress_index{i}'] = df[f'vpdmax_mean_{i}'] / (df[f'ppt_sum_{i}'] + 0.1)
df['local_relief'] = df['elev_mean'] - df['elev_min']

df['total_relief_log'] = np.log1p(df['total_relief'])

# # Example interactions
# Slope × aspect components using precomputed sine/cosine
df['slope_x'] = df['slope_mean'] * df['aspect_mean_sin']   # east-west component
df['slope_y'] = df['slope_mean'] * df['aspect_mean_cos']   # north-south component

In [20]:
df

,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_x,slope_y,year,evi_smooth_mean_4,evi_smooth_mean_5,evi_smooth_mean_6,evi_smooth_mean_7,evi_smooth_mean_8,evi_smooth_mean_9,evi_smooth_mean_10,mcari2_smooth_mean_4,mcari2_smooth_mean_5,mcari2_smooth_mean_6,mcari2_smooth_mean_7,mcari2_smooth_mean_8,mcari2_smooth_mean_9,mcari2_smooth_mean_10,ndvi_smooth_mean_4,ndvi_smooth_mean_5,ndvi_smooth_mean_6,ndvi_smooth_mean_7,ndvi_smooth_mean_8,ndvi_smooth_mean_9,ndvi_smooth_mean_10,ndwi_smooth_mean_4,ndwi_smooth_mean_5,ndwi_smooth_mean_6,ndwi_smooth_mean_7,ndwi_smooth_mean_8,ndwi_smooth_mean_9,ndwi_smooth_mean_10,rendvi_smooth_mean_4,rendvi_smooth_mean_5,rendvi_smooth_mean_6,rendvi_smooth_mean_7,rendvi_smooth_mean_8,rendvi_smooth_mean_9,rendvi_smooth_mean_10,savi_smooth_mean_4,savi_smooth_mean_5,savi_smooth_mean_6,savi_smooth_mean_7,savi_smooth_mean_8,savi_smooth_mean_9,savi_smooth_mean_10,evi_smooth_slope_4,evi_smooth_slope_5,evi_smooth_slope_6,evi_smooth_slope_7,evi_smooth_slope_8,evi_smooth_slope_9,evi_smooth_slope_10,mcari2_smooth_slope_4,mcari2_smooth_slope_5,mcari2_smooth_slope_6,mcari2_smooth_slope_7,mcari2_smooth_slope_8,mcari2_smooth_slope_9,mcari2_smooth_slope_10,ndvi_smooth_slope_4,ndvi_smooth_slope_5,ndvi_smooth_slope_6,ndvi_smooth_slope_7,ndvi_smooth_slope_8,ndvi_smooth_slope_9,ndvi_smooth_slope_10,ndwi_smooth_slope_4,ndwi_smooth_slope_5,ndwi_smooth_slope_6,ndwi_smooth_slope_7,ndwi_smooth_slope_8,ndwi_smooth_slope_9,ndwi_smooth_slope_10,rendvi_smooth_slope_4,rendvi_smooth_slope_5,rendvi_smooth_slope_6,rendvi_smooth_slope_7,rendvi_smooth_slope_8,rendvi_smooth_slope_9,rendvi_smooth_slope_10,savi_smooth_slope_4,savi_smooth_slope_5,savi_smooth_slope_6,savi_smooth_slope_7,savi_smooth_slope_8,savi_smooth_slope_9,savi_smooth_slope_10,evi_smooth_std_4,evi_smooth_std_5,evi_smooth_std_6,evi_smooth_std_7,evi_smooth_std_8,evi_smooth_std_9,evi_smooth_std_10,mcari2_smooth_std_4,mcari2_smooth_std_5,mcari2_smooth_std_6,mcari2_smooth_std_7,mcari2_smooth_std_8,mcari2_smooth_std_9,mcari2_smooth_std_10,ndvi_smooth_std_4,ndvi_smooth_std_5,ndvi_smooth_std_6,ndvi_smooth_std_7,ndvi_smooth_std_8,ndvi_smooth_std_9,ndvi_smooth_std_10,ndwi_smooth_std_4,ndwi_smooth_std_5,ndwi_smooth_std_6,ndwi_smooth_std_7,ndwi_smooth_std_8,ndwi_smooth_std_9,ndwi_smooth_std_10,rendvi_smooth_std_4,rendvi_smooth_std_5,rendvi_smooth_std_6,rendvi_smooth_std_7,rendvi_smooth_std_8,rendvi_smooth_std_9,rendvi_smooth_std_10,savi_smooth_std_4,savi_smooth_std_5,savi_smooth_std_6,savi_smooth_std_7,savi_smooth_std_8,savi_smooth_std_9,savi_smooth_std_10,ndvi_smooth_sos_month,evi_smooth_sos_month,ndwi_smooth_sos_month,savi_smooth_sos_month,rendvi_smooth_sos_month,mcari2_smooth_sos_month,ndvi_smooth_season_length,evi_smooth_season_length,ndwi_smooth_season_length,savi_smooth_season_length,rendvi_smooth_season_length,mcari2_smooth_season_length,ndvi_smooth_senescence_rate,evi_smooth_senescence_rate,ndwi_smooth_senescence_rate,savi_smooth_senescence_rate,rendvi_smooth_senescence_rate,mcari2_smooth_senescence_rate,frost_days_4,frost_days_5,frost_days_6,frost_days_7,frost_days_8,frost_days_9,frost_days_10,gdd_sum_4,gdd_sum_5,gdd_sum_6,gdd_sum_7,gdd_sum_8,gdd_sum_9,gdd_sum_10,ppt_avg_4,ppt_avg_5,ppt_avg_6,ppt_avg_7,ppt_avg_8,ppt_avg_9,ppt_avg_10,ppt_max_4,ppt_max_5,ppt_max_6,ppt_max_7,ppt_max_8,ppt_max_9,ppt_max_10,ppt_sum_4,ppt_sum_5,ppt_sum_6,ppt_sum_7,ppt_sum_8,ppt_sum_9,ppt_sum_10,ppt_var_4,ppt_var_5,ppt_var_6,ppt_var_7,ppt_var_8,ppt_var_9,ppt_var_10,tmax_max_4,tmax_max_5,tmax_max_6,tmax_max_7,tmax_max_8,tmax_max_9,tmax_max_10,tmax_mean_4,tmax_mean_5,tmax_mean_6,tmax_mean_7,tmax_mean_8,tmax_mean_9,tmax_mean_10,tmax_min_4,tmax_min_5,tmax_min_6,tmax_min_7,tmax_min_8,tmax_min_9,tmax_

In [21]:
df.to_pickle('data/df.pkl')